In [9]:
#!pip install gdown
#!pip install chromadb
!pip install -qU pypdf

In [45]:
import os
import getpass
import gdown
import chromadb

from dotenv import load_dotenv
from uuid import uuid4

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma #https://python.langchain.com/docs/integrations/vectorstores/chroma/
from langchain_community.document_loaders import PyPDFLoader

### Downloading archives from drive folder

In [60]:
from dotenv import dotenv_values
config = dotenv_values("/home/lgb/Documentos/Cursos/RAG-project/.env")

url = config["DRIVE_FOLDER_URL"]
docs_dir = config["DOCS_DIR"]

In [ ]:
gdown.download_folder(url, quiet=True, use_cookies=False)

### Embed documents

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_base="placeholder",
                              model = "bge-m3",
                              api_key = config["llm_key"])

In [58]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [23]:
persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(client=persistent_client,
                                  collection_name="collection_name",
                                  embedding_function=embeddings,)

/home/lgb/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:33<00:00, 2.48MiB/s]


In [62]:
documents = []
document_id = 1

for file_name in os.listdir(docs_dir):
    if file_name.endswith(".pdf"):
        file_path = os.path.join(docs_dir, file_name)
        loader = PyPDFLoader(file_path)
        
        async for page in loader.alazy_load():
            documents.append(Document(
                page_content=page.page_content,
                metadata=page.metadata,
                id=document_id,
            ))
            document_id += 1

uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['20e6cd31-bfe6-4ee0-8c61-b4435c037382',
 'ed067909-8ac1-4179-bb1e-bf6124dbee3f',
 '2d79eb22-1341-4a46-8147-642be6d06de0',
 '32d7f63b-fab5-427d-86f9-83b93fc3538f',
 'c6cfabd8-6e19-42ea-bf02-b16d55a2a244',
 '23f344b3-d94d-4ec6-a134-a4759a42c68a',
 'e60fcf98-ec54-4124-a0b4-7a2827fcc0a6',
 '82f1e973-7eee-4b8d-bb5e-387938da74b7',
 'd925a82b-5d46-4e67-80ba-b4e5ab70d825',
 '899c4063-49d3-4ca6-963e-c0cb45dd6818',
 '19e64f58-6c21-4f1f-8c6b-f7d047a1dea4',
 'df749388-5763-4a43-bb12-e1cd8e6e4846',
 'bac584bd-e6a8-4fd1-8777-2bd68629e524',
 'a3c023e0-73e7-467c-bdc0-8c1faf5ab893',
 '0856d19a-79be-47c1-8f9e-fa79fd81b8b2',
 '0e1adf16-2d91-418a-834e-8edb61a16995',
 '9a773d25-71b6-435e-b584-6708c2a2b340',
 'fade63f5-f26f-4a68-8595-077ac792e77b',
 'c66c46ca-d29e-4a3a-b7d9-594efece4d61',
 '780fffb2-b4f9-41fa-accc-f40d78e7e8a4',
 '5904a218-3047-4507-a0fe-822994a92637',
 '00cbdbc7-5f67-447c-a281-4889c471e3e0',
 'fa5aa37f-406b-4f7b-938f-b4b72dda57f8',
 'a252200e-12cc-4cb8-a175-99cf61fdba91',
 '6ff0f333-4ebc-

In [63]:
print(f"{len(documents)} páginas carregadas e indexadas.")

2811 páginas carregadas e indexadas.


### Query documents

In [27]:
results = vector_store.similarity_search(
    "fale sobre DISPOSIÇÕES PRELIMINARES",
    k=1,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* RESOLUÇÃO ANP Nº 882, DE 27 DE JULHO DE 2022 - DOU DE 28.07.2022
Estabelece o procedimento para a comunicação
de incidentes e o envio de relatórios de
investigação pelos operadores de contrato de
exploração e produção de petróleo e gás natural
e pelas empresas autorizadas a exercer as
atividades da indústria do petróleo, gás natural
e biocombustíveis.
A DIRETORIA DA AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E
BIOCOMBUSTÍVEIS - ANP, no exercício das atribuições conferidas pelo art. 65 do
Regimento Interno, aprovado pela Portaria ANP nº  
265
, de 10 de setembro de
2020, e pelo art. 7º do Anexo I do Decreto nº  
2.455
, de 14 de janeiro de 1998,
tendo em vista o disposto na Lei nº 
9.478
, de 6 de agosto de 1997, na Lei nº 
9.966
,
de 28 de abril de 2000 e no Decreto nº  
4.136
, de 20 de fevereiro de 2002,
considerando o que consta do Processo nº 48610.009186/2017-67 e as
deliberações tomadas na 1.096ª Reunião de Diretoria, realizada em 26 de julho de
2022, RESOLVE:
CAPÍTULO I
DISPOSI